# Code to compute predictions and the confusion matrix

In [ ]:
import os
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import seaborn as sns
from keras.layers import Layer
from keras.utils import CustomObjectScope
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras import initializers

print(tf.__version__)

In [ ]:
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
class LayerScale(Layer):
    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, _):
        self.gamma = self.add_weight(
            shape=(self.projection_dim,),
            initializer=initializers.Constant(self.init_values),
            trainable=True,
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

In [ ]:
DATA_PATH = '/kaggle/input/training-data-final/training_data_final'

In [ ]:
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 64

IMG_SIZE = (96,96)
IMG_SHAPE = IMG_SIZE + (3,)
N_CLASSES = 2

training_set = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    labels='inferred',
    label_mode='binary',
    class_names = None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(96,96),
    shuffle=True,
    seed=seed,
    validation_split=VALIDATION_SPLIT,
    subset='training'
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    labels='inferred',
    label_mode='binary',
    class_names = None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(96,96),
    shuffle=True,
    seed=seed,
    validation_split=VALIDATION_SPLIT,
    subset='validation'
)

In [ ]:
MODEL_PATH = '/kaggle/input/basso093model/SubmissionModel'

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH, custom_objects={'LayerScale': LayerScale}, safe_mode=False)

predictions = []
labels = []
for x, y in validation_set:
    pred_batch = model.predict(x)
    pred_batch = pred_batch.reshape(-1)
    predictions.extend(pred_batch)
    labels.extend(y.numpy())

predictions = np.array(predictions)
binary_predictions = np.where(predictions > 0.5, 1, 0)
cm = confusion_matrix(labels, binary_predictions)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 7))

sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', annot_kws={"size": 16})

plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('/kaggle/working/confusion_matrix.png', dpi=300)
plt.show()

In [ ]:
from IPython.display import FileLink
FileLink(r'confusion_matrix.png')

# With TTA

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH, custom_objects={'LayerScale': LayerScale}, safe_mode=False)

def tta_prediction(image):
    TTA_STEPS = 10
    BATCH_SIZE = 32

    test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=[0.90, 1.10],
        vertical_flip=True,
        horizontal_flip=True,
        fill_mode='reflect')

    yhats = []

    for i in range(TTA_STEPS):
        preds = model.predict(test_gen.flow(image, batch_size=BATCH_SIZE, shuffle=False))
        yhats.append(preds)

    mean_pred = tf.reduce_mean(yhats, axis=0)
    tta_pred = tf.cast(mean_pred > 0.5, tf.int32)
    tta_pred_reshaped = tf.reshape(tta_pred, [-1])

    return tta_pred_reshaped

predictions = []
labels = []
for x, y in validation_set:
    pred_batch = tta_prediction(x)
    predictions.extend(pred_batch)
    labels.extend(y.numpy())

predictions = np.array(predictions)
binary_predictions = np.where(predictions > 0.5, 1, 0)
cm = confusion_matrix(labels, binary_predictions)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 7))

sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', annot_kws={"size": 16})

plt.xlabel('Predicted', fontsize=14)
plt.ylabel('Actual', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('/kaggle/working/tta_confusion_matrix.png', dpi=300)
plt.show()

In [ ]:
from IPython.display import FileLink
FileLink(r'tta_confusion_matrix.png')